# RewardFunction16
```
Key Elements:
1. New base function
2. Has base reward, reward progress relative to steps, reward speed based on curvature of track, and gives a small 
   reward for being left of center
3. Curvature calculations based on vector calculations
```

In [3]:
def reward_function(params):
    '''
    Base model where curvature is determined by vector calculations
    '''
   
    # Read input parameters
    #x_car = params['x']
    #y_car = params['y']
    on_track = params['all_wheels_on_track']
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    progress = params['progress']/100
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    steering = abs(params['steering_angle']) # Only need the absolute steering angle
    speed = params['speed']
    steps = params['steps']
    is_left_of_center = params['is_left_of_center']
    
    REWARD_MIN = 0.1
    REWARD_MAX = 100
    TARGET_STEPS = 300
    
    trackStraight = True
    
    # Calculate 3 marks that are farther and father away from the center line
    marker_1 = 0.25 * track_width
    marker_2 = 0.4 * track_width
    marker_3 = 0.6 * track_width
    
    # Set Base Reward
    if not on_track: # Fail them if off Track
        return REWARD_MIN
    elif progress == 1:
        #the lap is complete. if we use more steps than TARGET_STEPS, the reward is lower
        return REWARD_MAX * TARGET_STEPS / steps
    else:        
        reward = REWARD_MAX * max(progress, 0.4)
    
    # Determine waypoints
    prev_pt = waypoints[closest_waypoints[0]] # closest waypoint behind car
    next_pt = waypoints[closest_waypoints[1]] # closest waypoint ahead of car
    nextnext_pt = waypoints[(closest_waypoints[1] + 1) % len(waypoints)] # waypoint after next_pt
    
    vector1 = [next_pt[0] - prev_pt[0], next_pt[1] - prev_pt[1]]
    vector2 = [nextnext_pt[0] - next_pt[0], nextnext_pt[1] - next_pt[1]]
    
    dist1 = (vector1[0]**2 + vector1[1]**2)**0.5    
    dist2 = (vector2[0]**2 + vector2[1]**2)**0.5 
    
    if (dist2 > 0 and dist1 > 0):
        #we will analyze the relative distances between waypoints
        #compute the cosine of the angle between the vectors connecting consecutive waypoints
        cosineAngle = abs(vector1[0]*vector2[0] + vector1[1]*vector2[1]) / (dist1 * dist2)
        #cosineAngle will be closer to 1 if the curve is mostly straight
        if cosineAngle < 0.99:
            #the track might be curved
            ratio = abs(dist2 - dist1)/(dist2 + dist1)
            #the track might be curved if these consecutive waypoints are relatively close together
            #if dist1 and dist2 are close to the same values, ratio will be very small.
            #use a cutoff of 0.2 to decide that the angle between waypoint vectors is noticable
            #so we'll look at the relative spacing between waypoints
            if ratio < 0.2:
                #this is the only way that we will consider the track curved
                trackStraight = False
        
    # Reward based on whether track is straight or curved
    if trackStraight:
        #track is straight
        reward += (speed**3)/3.75 + (progress*900/(((steps**2) + 1)**.5))
        if is_left_of_center:
            reward += 1
    else:
        #track is curved
        if distance_from_center <= marker_2:
            reward += 5
        elif distance_from_center <= marker_3:
            reward += 3
        reward += ((500*progress)/(((steps**2) + 1)**.5))
    
    if is_left_of_center:
        reward += 1
    
    return float(reward)
